# Convert annotations of MakeML dataset to YOLO format

In [75]:
#import libraries
import glob
import os
import xml.etree.ElementTree as ET
import pandas as pd

Define function to convert annotations

In [45]:
def convert (xmin, xmax, ymin, ymax, img_width, img_height):
    xcenter = (xmin + xmax) / img_width / 2
    ycenter = (ymin + ymax) / img_height / 2
    w = (xmax - xmin) / img_width
    h = (ymax - ymin) / img_height
    return xcenter, ycenter, w, h

Define the path to the directory containing the XML annotations and the path to the directory to write the YOLO anonotations. Create the directory if it does not exist

In [46]:
xml_ann_path = "MakeML/annotations"
yolo_ann_path = xml_ann_path + "_yolo/"

try:
    os.mkdir(yolo_ann_path)
    print("Directory '% s' created" % yolo_ann_path)
except OSError as error:
    print("Directory already exists")

Directory already exists


Create (or open) a csv file to save the annotations information (for easier manipulation and further analysis using pandas) and add header

In [70]:
csv_file = "gt_MakeML_yolo.csv"
csv_out = open(csv_file, "w")
csv_out.write('file,xcenter,ycenter,width,height,class\n')

40

Define the classes in the dataset. They must respect the order of reclassification (0-trafficlight, 1-stop, 2-speedlimit, 3-crosswalk)

In [71]:
classes = ['trafficlight', 'stop', 'speedlimit', 'crosswalk']

Read the XML annotation files, extract the necessary information, convert coordinates and classes, write to the corresponding YOLO annotation file, and write to the csv file 

In [ ]:
for filename in glob.glob( xml_ann_path + "/*.xml"): 
        
    #get the basename (without extension) of the file
    basename = os.path.basename(filename)
    basename = os.path.splitext(basename)[0]
    
    #open the file to read
    ann_in = open(filename, "r")
    #open (create) the file to write
    ann_out = open(path + basename + ".txt", "w")
    
    tree = ET.parse(ann_in)
    root = tree.getroot()
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)
    
    for obj in root.iter('object'):
        cls = obj.find('name').text
        class_id = classes.index(cls)
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        xmax = int(bbox.find('xmax').text)
        ymin = int(bbox.find('ymin').text)
        ymax = int(bbox.find('ymax').text)
        
        xcenter, ycenter, w, h = convert(xmin, xmax, ymin, ymax, img_width, img_height)
        csv_out.write(basename + ".png," + str(xcenter) + "," + str(ycenter) + "," + str(w) + "," + str(h) + "," + str(class_id) + "\n")
        ann_out.write(str(class_id) + " " + str(xcenter) + " " + str(ycenter) + " " + str(w) + " " + str(h) + "\n")
    
    ann_out.close()
    
csv_out.close()

In [74]:
df = pd.read_csv("gt_MakeML_yolo.csv")
df.head(10)

,file,xcenter,ycenter,width,height,class
0,road0.png,0.573034,0.367500,0.411985,0.425000,0
1,road1.png,0.515000,0.607774,0.260000,0.770318,0
2,road10.png,0.437500,0.498127,0.345000,0.973783,0
3,road100.png,0.497500,0.429870,0.820000,0.833766,2
4,road101.png,0.733750,0.502500,0.492500,0.935000,2
5,road102.png,0.350000,0.529963,0.515000,0.775281,2
6,road103.png,0.482500,0.493355,0.520000,0.820598,2
7,road104.png,0.490000,0.493132,0.740000,0.909341,2
8,road105.png,0.513333,0.653750,0.846667,0.577500,2
9,road106.png,0.494382,0.470000,0.232210,0.190000,2
